Grab this for later

In [ ]:
try:
  import google.colab
  IN_COLAB = True
  WORKDIR = "/content"
except:
  IN_COLAB = False
  WORKDIR = "/home/jim/Playing/tessupd"

In [ ]:
if IN_COLAB:
  !wget https://github.com/jimregan/tesseract-gle-uncial/releases/download/v0.1beta2/gle_uncial.traineddata

Install dependencies

In [ ]:
if IN_COLAB:
  !apt-get install libicu-dev libpango1.0-dev libcairo2-dev libleptonica-dev

Clone, compile and set up Tesseract

In [ ]:
if IN_COLAB:
  !git clone https://github.com/tesseract-ocr/tesseract

In [ ]:
import os
os.chdir('tesseract')

In [ ]:
!sh autogen.sh

In [ ]:
!./configure --disable-graphics


In [ ]:
!make -j 8
!make install
!ldconfig
!make training
!make training-install

Grab some things to scrape the RIA corpus

In [ ]:
import os
os.chdir(WORKDIR)
!git clone https://github.com/jimregan/tesseract-gle-uncial/

Cloning into 'tesseract-gle-uncial'...
remote: Enumerating objects: 1435, done.
remote: Total 1435 (delta 0), reused 0 (delta 0), pack-reused 1435
Receiving objects: 100% (1435/1435), 200.26 MiB | 29.17 MiB/s, done.
Resolving deltas: 100% (656/656), done.
Checking out files: 100% (632/632), done.


In [ ]:
!apt-get install lynx

Scrape the RIA corpus

In [ ]:
! for i in A B C D E F G H I J K L M N O P Q R S T U V W X Y Z;do lynx -dump "http://corpas.ria.ie/index.php?fsg_function=1&fsg_page=$i" |grep http://corpas.ria.ie|awk '{print $NF}' >> list;done

In [ ]:
!grep 'function=3' list |sort|uniq|grep corpas.ria|sed -e 's/function=3/function=5/' > input

In [ ]:
!wget -x -c -i input

In [ ]:
!mkdir text
!for i in corpas.ria.ie/*;do id=$(echo $i|awk -F'=' '{print $NF}');cat $i | perl {WORKDIR}/tesseract-gle-uncial/scripts/extract-ria.pl > text/$id.txt;done

Get the raw corpus in a single text file

In [ ]:
!cat text/*.txt|tr -d '\r'|grep -v '^$' > ria-raw.txt


Compress the raw text; this can be downloaded through the file browser on the left, so the scraping steps can be skipped in future

In [ ]:
!gzip ria-raw.txt

...and can be re-added using the upload feature in the file browser

In [ ]:
!gzip -d ria-raw.txt.gz

This next part is so I can update the langdata files

In [ ]:
import os
os.chdir(WORKDIR)
!git clone https://github.com/tesseract-ocr/langdata

Cloning into 'langdata'...
remote: Enumerating objects: 2164, done.
remote: Total 2164 (delta 0), reused 0 (delta 0), pack-reused 2164
Receiving objects: 100% (2164/2164), 400.15 MiB | 21.02 MiB/s, done.
Resolving deltas: 100% (382/382), done.
Checking out files: 100% (1022/1022), done.


In [ ]:
!cat ria-raw.txt | perl {WORKDIR}/tesseract-gle-uncial/scripts/toponc.pl > ria-ponc.txt

In [ ]:
!mkdir genwlout

In [ ]:
!perl {WORKDIR}/tesseract-gle-uncial/scripts/genlangdata.pl -i ria-ponc.txt -d genwlout -p gle_uncial

In [ ]:
import os
os.chdir(f'{WORKDIR}/genwlout')
!for i in gle_uncial.word.bigrams gle_uncial.wordlist gle_uncial.numbers gle_uncial.punc; do cat $i.unsorted | awk -F'\t' '{print $1}' | sort | uniq > $i.sorted;done
!for i in gle_uncial.word.bigrams gle_uncial.wordlist gle_uncial.numbers gle_uncial.punc; do cat $i.sorted {WORKDIR}/langdata/gle_uncial/$i | sort | uniq > $i;done

cat: gle_uncial.word.bigrams.sorted: No such file or directory
cat: gle_uncial.wordlist.sorted: No such file or directory
cat: gle_uncial.numbers.sorted: No such file or directory
cat: gle_uncial.punc.sorted: No such file or directory


In [ ]:
!for i in gle_uncial.word.bigrams gle_uncial.wordlist gle_uncial.numbers gle_uncial.punc; do cp $i {WORKDIR}/langdata/gle_uncial/;done

Grab the fonts

In [ ]:
import os
os.chdir(WORKDIR)
!mkdir fonts
os.chdir('fonts')
!wget -i /content/tesseract-gle-uncial/fonts.txt

In [ ]:
!for i in *.zip; do unzip $i;done

Generate

In [ ]:
os.chdir(WORKDIR)
!mkdir unpack
!combine_tessdata -u {WORKDIR}/gle_uncial.traineddata unpack/gle_uncial.

In [ ]:
os.chdir('unpack')
!for i in gle_uncial.word.bigrams gle_uncial.wordlist gle_uncial.numbers gle_uncial.punc; do cp {WORKDIR}/genwlout/$i .;done

In [ ]:
!wordlist2dawg gle_uncial.numbers  gle_uncial.lstm-number-dawg gle_uncial.lstm-unicharset
!wordlist2dawg gle_uncial.punc  gle_uncial.lstm-punc-dawg gle_uncial.lstm-unicharset
!wordlist2dawg gle_uncial.wordlist  gle_uncial.lstm-word-dawg gle_uncial.lstm-unicharset

Loading unicharset from 'gle_uncial.lstm-unicharset'
Reading word list from 'gle_uncial.numbers'
Reducing Trie to SquishedDawg
Writing squished DAWG to 'gle_uncial.lstm-number-dawg'
Loading unicharset from 'gle_uncial.lstm-unicharset'
Reading word list from 'gle_uncial.punc'
Reducing Trie to SquishedDawg
Writing squished DAWG to 'gle_uncial.lstm-punc-dawg'
Loading unicharset from 'gle_uncial.lstm-unicharset'
Reading word list from 'gle_uncial.wordlist'
Reducing Trie to SquishedDawg
Writing squished DAWG to 'gle_uncial.lstm-word-dawg'


In [ ]:
!rm gle_uncial.numbers gle_uncial.word.bigrams gle_uncial.punc gle_uncial.wordlist

In [ ]:
os.chdir(WORKDIR)
!mv gle_uncial.traineddata gle_uncial.traineddata.orig
!combine_tessdata unpack/gle_uncial.

Combining tessdata files
Output unpack/gle_uncial.traineddata created successfully.
Version string:4.00.00alpha
17:lstm:size=4291340, offset=192
18:lstm-punc-dawg:size=890, offset=4291532
19:lstm-word-dawg:size=5767506, offset=4292422
20:lstm-number-dawg:size=226, offset=10059928
21:lstm-unicharset:size=9938, offset=10060154
22:lstm-recoder:size=1147, offset=10070092
23:version:size=12, offset=10071239


In [ ]:
os.chdir(WORKDIR)
!bash {WORKDIR}/tesseract/src/training/tesstrain.sh

USAGE: tesstrain.sh
     --exposures EXPOSURES      # A list of exposure levels to use (e.g. "-1 0 1").
     --fontlist FONTS           # A list of fontnames to train on.
     --fonts_dir FONTS_PATH     # Path to font files.
     --lang LANG_CODE           # ISO 639 code.
     --langdata_dir DATADIR     # Path to tesseract/training/langdata directory.
     --linedata_only            # Only generate training data for lstmtraining.
     --output_dir OUTPUTDIR     # Location of output traineddata file.
     --overwrite                # Safe to overwrite files in output_dir.
     --run_shape_clustering     # Run shape clustering (use for Indic langs).
     --maxpages                 # Specify maximum pages to output (default:0=all)
     --save_box_tiff            # Save box/tiff pairs along with lstmf files.
     --xsize                    # Specify width of output image (default:3600)

  OPTIONAL flag for specifying directory with user specified box/tiff pairs.
  Files should be named sim

In [ ]:
!text2image --fonts_dir fonts --list_available_fonts

  0: Humor Sans
  1: Liberation Mono
  2: Liberation Mono Bold
  3: Liberation Mono Bold Italic
  4: Liberation Mono Italic
  5: Liberation Sans
  6: Liberation Sans Bold
  7: Liberation Sans Bold Italic
  8: Liberation Sans Italic
  9: Liberation Sans Narrow Bold Condensed
 10: Liberation Sans Narrow Bold Italic Condensed
 11: Liberation Sans Narrow Condensed
 12: Liberation Sans Narrow Italic Condensed
 13: Liberation Serif
 14: Liberation Serif Bold
 15: Liberation Serif Bold Italic
 16: Liberation Serif Italic


In [ ]:
!cat genwlout/gle_uncial.wordlist.unsorted|awk -F'\t' '{print $2 "\t" $1'}|sort -nr > freqlist

In [ ]:
!cat freqlist|awk -F'\t' '{print $2}'|grep -v '^$' > wordlist

In [ ]:
!cat ria-ponc.txt|sort|uniq|head -n 400000 > gle_uncial.training_text

In [ ]:
!cp unpack/gle_uncial.traineddata /usr/share/tesseract-ocr/4.00/tessdata

In [ ]:
!echo 'jK}' >> gle_uncial.training_text
!cp gle_uncial.training_text langdata/gle_uncial/

In [ ]:
!mkdir output

In [ ]:
!wget https://github.com/tesseract-ocr/tessdata/blob/master/eng.traineddata?raw=true -O /usr/share/tesseract-ocr/4.00/tessdata/eng.traineddata

In [ ]:
!bash tesseract/src/training/tesstrain.sh --fonts_dir fonts --lang gle_uncial --linedata_only --noextract_font_properties --langdata_dir langdata --tessdata_dir /usr/share/tesseract-ocr/4.00/tessdata --output_dir output

In [ ]:
!mkdir -p /usr/share/tesseract-ocr/4.00/tessdata